In [25]:
''' 

Ausgangslage: Programmierung kopiert von https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb


Die Abhängigkeiten der Programmierung sind angepasst, so dass sie ins VOEBB-Forum geladen werden können.

Ziel der Programmierung ist es:
ich extrahiere die ISBNs eines Metadatesatzes als Array, gebe die ISBNs nacheinander in die Abfrage ein 
und erhalte am Schluss den Metadatensatz angereichert mit dem Inhaltsverzeichnis in der Spalte „Inhaltsverzeichnis“.

Vom Exception Handling werden folgende Dinge abgedeckt:

es gibt kein Medium mit der ISBN auf den DNB-Seiten (ausländisches Medium)
es gibt kein Inhaltsverzeichnis zu dem Medium auf den DNB-Seiten
es gibt Verbindungsprobleme bei der Stapelverarbeitung. In dem Fall versucht er insgesamt 5 Mal in Abstand von 8s.


Die Fuktionen dnb_sru und parse_record sind unverändert aus der DNB-Programmierung übernommen. 

ToDo:
Copywarning am Ende adressieren
Die Speicherung als Excel funktioniert nicht. Speicherdatenformat überdenken - wie geht der Datenflow weiter?


'''


' \nAbhängigkeiten für das Forum anpassen\n\n\nAusgangslage: Programmierung kopiert von https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb\nthank you dnb...\n\nDie Abhängigkeiten der Programmierung sind angepasst, so dass sie ins VOEBB-Forum geladen werden können.\n\nZiel der Programmierung ist es:\nich extrahiere die ISBNs eines Metadatesatzes als Array, gebe die ISBNs nacheinander in die Abfrage ein \nund erhalte am Schluss den Metadatensatz angereichert mit dem Inhaltsverzeichnis in der Spalte „Inhaltsverzeichnis“.\n\nVom Exception Handling werden folgende Dinge abgedeckt:\n\nes gibt kein Medium mit der ISBN auf den DNB-Seiten (ausländisches Medium)\nes gibt kein Inhaltsverzeichnis zu dem Medium auf den DNB-Seiten\nes gibt Verbindungsprobleme bei der Stapelverarbeitung. In dem Fall versucht er insgesamt 5 Mal in Abstand von 8s.\n\n\nDie Fuktionen dnb_sru und parse_record sind unverändert aus der DNB-Programmierung übernom

In [26]:
# Stand 22. Okt. 2025

In [27]:


import re
import requests
import unicodedata
from lxml import etree
import pandas as pd
from bs4 import BeautifulSoup as soup
from collections import Counter
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datetime import datetime


In [28]:
#laden der mit den inhaltsverzeichnissen anzureichernden Datei
#

df = pd.read_excel('20251010_dummy_titelliste_2.xlsx')

In [29]:
# hinzufügen der Splate "inhaltsverzeichnis"

df['inhaltsverzeichnis']=pd.Series(dtype="str")
#df['links']=pd.Series(dtype="str")

In [30]:
# array mit ISBN-Werten erstellen
isbn=df['ISBN']
for i in isbn:
    print(i)

978-3-0358-0042-5
0-415-17473-2
978-3-518-22511-0
978-3-942449-36-6
978-3-8376-2446-5
978-3-7416-2280-9
978-3-7416-1018-9
978-3-7416-0481-2
978-3-86201-588-7
978-3-86201-326-5
978-3-7416-2003-4
978-3-949866-04-3
978-3-608-98658-7
978-3-946807-54-4
978-3-95972-375-6
978-3-7074-5294-5
978-3-98921-091-2


In [31]:
# Funktion kopiert aus einem DNB Workshop im Netz:
#https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb
# scraped den zu der isbn gehörenden kompletten datensatz von der internetseite

def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '10',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 10:
        
        return records
    
    else:
        
        num_results = 10
        i = 11
        while num_results == 10:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=10
            num_results = len(new_records)
            
        return records

In [32]:
# Funktion kopiert aus einem DNB Workshop im Netz:
#https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb
# scraped den link zum inhaltsverzeichnis aus dem record datensatz
# hier kannst du dann auch andere Marc-Felder wie SChlagwort adressieren

def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    descriptions = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = '3']", namespaces=ns)
    links = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        description1 = descriptions[0].text
    except:
        description1 = "unknown"
        
    try: 
        description2 = descriptions[1].text
    except:
        description2 = "unknown"
        
        
    if description1 == "Inhaltsverzeichnis": 
        link = links[0].text
    elif description2 == "Inhaltsverzeichnis": 
        link = links[1].text
    else:
        link = "unknown"

    
        
        
    meta_dict = {"link":link + '/text'}
    
    return meta_dict

In [33]:
# initiert die Variable mit der der Index des Dataframes in der untenstehenden for-Schleife 
#trotz exception handling adressiert werden kann
t=-1 

# falls die Verbindung zur Webseite abgelehnt wird, ist das eine Funktion die die 
#Wiederholung der Abfrage organisiert

session = requests.Session()

retries = Retry(
    total=5,                    # total attempts
    backoff_factor=8,           # wait 1, 2, 4, 8, ... seconds between retries
    status_forcelist=[500, 502, 503, 504],  # retry on these HTTP codes
    allowed_methods=["GET", "POST"]
)

In [34]:
for i in isbn:
    records = dnb_sru(i)
    inhaltsverzeichnis = [parse_record(record) for record in records]
    x = pd.DataFrame(inhaltsverzeichnis)

    # "t" nimmt den Index des Dataframes in den das Inhaltsverzeichnis geladen werden soll
    # mit in die for-Schleife (s.u.)
    t += 1

    # wenn der Titel kein Inhaltsverzeichnis hat
    if "unknown/text" in str(x):
        continue

    # wenn der titel nicht in der dnb vorkommt (ausländischer titel)
    if inhaltsverzeichnis == []:
        continue
    
    # url speichert den Link zum Inhaltsverzeichnis
    url=x['link'].iloc[0]
    
    # Filename speichert das Inhaltsverzeichnis unter isbn +'.txt'
    filename = str(i) + ".txt"
    
    # folgende Angaben organisieren die wiederholte Anfrage an die DNB-Seite, wenn die Verbindung 
    # abgelehnt wurde und den Download der Datei
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        with open(filename, "wb") as f:
            f.write(r.content)
            print("Download successful:", filename)
    except requests.exceptions.RequestException as e:
        print("Download failed:", e)
    
    # lädt die Donwload-Datei und speichert sie in den Dataframe mit den Metadaten.
    z=pd.read_fwf(filename)   # gespeicherte datei öffnen und in der variablen "z"speichern
    df['inhaltsverzeichnis'].iloc[t]= z   # z in richigen Reihe und Spalte des Dataframe speichern
    


    

Download successful: 978-3-942449-36-6.txt


/home/elsa_swift/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Download successful: 978-3-7416-2280-9.txt


/home/elsa_swift/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Download successful: 978-3-7416-0481-2.txt


...

/home/elsa_swift/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Download successful: 978-3-95972-375-6.txt


/home/elsa_swift/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Download successful: 978-3-7074-5294-5.txt


/home/elsa_swift/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [35]:
# angaben korrekt?

df['inhaltsverzeichnis']

0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   ...
4                                                   NaN
5                  INNERES LICHT Unnamed: 1  Unnamed...
6                                                   NaN
7                           NIEMANDSLAND: (UN)RECHT ...
8                                                   NaN
9                                                   NaN
10       SCHRECKLICH Kapitel 1 Terrible, Part 1 of3 ...
11                                                  NaN
12                                                In...
13                                                  NaN
14                                                  ...
15         FRANZIS IDEEN für die Pflanzen
0         ...
16                                                  NaN
Name: inhaltsverzeichnis, dtype: object

In [37]:
# dataframe als excel speichern
# funktioniert nicht - excel scheint auch nicht das richige Speicherformat.
# Wie geht der Datenflow weiter?


# aktuelles Datum im Format JJJJ-MM-TT
#heute = datetime.now().strftime("%Y-%m-%d")




#df.to_excel(heute+'_inhaltverzeichnisse_anreichern.xlsx')